#Pre-processing the data

Import the libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
dataset=pd.read_csv("Churn_Modelling.csv")
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Making the matrix of features and seperating the dependent variable

In [0]:
X=dataset.iloc[:,3:13].values
y=dataset.iloc[:,-1].values

Encoding the categorical variables

In [0]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])
# so we get the encoded values but there might be problems, that is why we do dummy coding

In [10]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

Dummy coding the categorical variables

In [0]:
#Dummy coding using OneHotEncoder
#X
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
X = transformer.fit_transform(X.tolist())
X = X.astype('float64')

# To avoid the dummy variable trap , we remove one  of the dummy variables 
X=X[:,1:]

In [12]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

Splitting the dataset into test set and training set

In [0]:
#splitting the dataset into a training set and a test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [14]:
X_train

array([[0.0000000e+00, 1.0000000e+00, 6.6700000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.6383064e+05],
       [1.0000000e+00, 0.0000000e+00, 4.2700000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 5.7098000e+04],
       [0.0000000e+00, 0.0000000e+00, 5.3500000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8563076e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.3800000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8142987e+05],
       [0.0000000e+00, 1.0000000e+00, 5.9000000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.4875016e+05],
       [1.0000000e+00, 0.0000000e+00, 6.2300000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1885526e+05]])

#Applying XGBoost to the training set and making predictions on the test set

Fitting the model

In [15]:
from xgboost import XGBClassifier
classifier=XGBClassifier()
#n_estimators is the number of trees
classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Making predictions on the test set

In [0]:
y_pred=classifier.predict(X_test)

# Evaluating the accuracy of the model 

Using confusion matrix

In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [21]:
cm

array([[1521,   74],
       [ 197,  208]])

Applying k-fold cross validation 

In [0]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=10)

In [23]:
accuracies.mean()

0.8625

In [24]:
accuracies.std()

0.01017042280340401

##**Notes**:
1.XGBoost is based on gradient boosting decision trees , so it is an excellent algorithm for classification.

2.We don't need to apply feature scaling in this model, so we can keep the interpretability of the dataset.

3.This algorithm has a very fast execution as compared to ANN but gives approximately the same result.
